In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

## Data Collection for Netflix, AmazonPrime,Disney+ and Hulu Movies from [justwatch.com](https://www.justwatch.com)

Web Scraping movies from streaming platfroms(Netflix,Amazon-Prime, Disney+ and Hulu) from a website 'justwatch.com'using selenium. The Movies scraped along with other information was about 1500+ and this would be used for further analysis

In [7]:
#create a function to scroll the webpage using selenium

def scroll_down(driver):
    # Scroll down to load more movies
    for i in range(50):  # adjust range based on how many times you want to scroll down
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(8)
        try:
            driver.find_element(By.CSS_SELECTOR, ".custom-infinite-scroll.custom-infinite-scroll--hidden-spinner").click()
        except:
            break

    SCROLL_PAUSE_TIME = 2
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height 

## Getting all Netflix Movies Data From 2013-2022

In [10]:
#create a Function to gather all the urls as well as the info needed(genres,titles,year,runtime etc)

def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/netflix/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

    # Get the IMDb rating
    def get_imdb_rating(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
            rating = imdb_tag.text.strip().split()[0]
        except AttributeError:
            rating = 'N/A'
        return rating
    rating = get_imdb_rating(url)
    ratings_list.append(rating)
    print(f"IMDb rating: {rating}")

    # Get the TV age rating
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    if age_rating_div:
        age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    else:
        age_rating = 'N/A'
    Age_Rating.append(age_rating)
    print(f"Age rating: {age_rating}")
    
    #get the movie genres
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    try:
        genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    except AttributeError:
        genre = 'N/A'
    genres.append(genre)
    print(f'Genres: {genre}')

    # Get the movie runtime
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    runtimes.append(runtime)
    print(f"Runtime: {runtime}")

    # Get the movie director
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    if director_div:
        director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
        directors.append(director_name)
    else:
        directors.append('N/A')
    print(f"Director: {director_name}\n")



Title: A Man Called Otto
Released year: 2022
IMDb rating: 7.4
Age rating: PG-13
Genres: Comedy, Drama
Runtime: 2h 6min
Director: Marc Forster

Title: Soft & Quiet
Released year: 2022
IMDb rating: 6.3
Age rating: R
Genres: Drama, Mystery & Thriller, Horror
Runtime: 1h 31min
Director: Beth de Araújo

Title: Above Suspicion
Released year: 2019
IMDb rating: 5.7
Age rating: R
Genres: Mystery & Thriller, Crime, Action & Adventure
Runtime: 1h 44min
Director: Phillip Noyce

Title: Bullet Train
Released year: 2022
IMDb rating: 7.3
Age rating: R
Genres: Mystery & Thriller, Comedy, Action & Adventure
Runtime: 2h 6min
Director: David Leitch

Title: The Glass Castle
Released year: 2017
IMDb rating: 7.1
Age rating: PG-13
Genres: Drama
Runtime: 2h 7min
Director: Destin Daniel Cretton

Title: Prisoners
Released year: 2013
IMDb rating: 8.1
Age rating: R
Genres: Mystery & Thriller, Crime, Drama
Runtime: 2h 33min
Director: Denis Villeneuve

Title: This Is the End
Released year: 2013
IMDb rating: 6.6
Age 

Title: The Good Nurse
Released year: 2022
IMDb rating: 6.8
Age rating: R
Genres: Drama, Mystery & Thriller, Crime
Runtime: 2h 1min
Director: Tobias Lindholm

Title: After Ever Happy
Released year: 2022
IMDb rating: 4.5
Age rating: R
Genres: Drama, Romance
Runtime: 1h 35min
Director: Castille Landon

Title: 2,215
Released year: 2018
IMDb rating: 7.7
Age rating: N/A
Genres: Documentary
Runtime: 1h 30min
Director: Nottapon Boonprakob

Title: Re/Member
Released year: 2022
IMDb rating: 5.1
Age rating: PG-13
Genres: Horror, Mystery & Thriller, Drama
Runtime: 1h 43min
Director: Eiichiro Hasumi

Title: The Platform
Released year: 2019
IMDb rating: 7.0
Age rating: N/A
Genres: Drama, Mystery & Thriller, Science-Fiction, Horror, Made in Europe
Runtime: 1h 35min
Director: Galder Gaztelu-Urrutia

Title: Nocturnal Animals
Released year: 2016
IMDb rating: 7.5
Age rating: R
Genres: Mystery & Thriller, Drama
Runtime: 1h 55min
Director: Tom Ford

Title: Furies
Released year: 2022
IMDb rating: 5.3
Age ra

Title: Blade of the 47 Ronin
Released year: 2022
IMDb rating: 4.7
Age rating: R
Genres: Action & Adventure, Drama, Fantasy, Mystery & Thriller
Runtime: 1h 47min
Director: Ron Yuan

Title: The Wretched
Released year: 2020
IMDb rating: 5.8
Age rating: N/A
Genres: Horror, Mystery & Thriller
Runtime: 1h 35min
Director: Drew T. Pierce

Title: Office Invasion
Released year: 2022
IMDb rating: 4.5
Age rating: R
Genres: Science-Fiction, Comedy, Action & Adventure, Horror
Runtime: 1h 52min
Director: Fred Wolmarans

Title: Fullmetal Alchemist: The Final Alchemy
Released year: 2022
IMDb rating: 5.4
Age rating: PG-13
Genres: Fantasy, Action & Adventure
Runtime: 2h 23min
Director: Fumihiko Sori

Title: For Jojo
Released year: 2022
IMDb rating: 4.0
Age rating: N/A
Genres: Drama
Runtime: 1h 37min
Director: Barbara Ott

Title: R.I.P.D. 2: Rise of the Damned
Released year: 2022
IMDb rating: 4.5
Age rating: PG-13
Genres: Action & Adventure, Crime, Fantasy, Science-Fiction, Western, Comedy
Runtime: 1h 42m

IMDb rating: 4.4
Age rating: PG
Genres: Comedy, Romance
Runtime: 1h 30min
Director: Alex Ranarivelo

Title: Phantom Thread
Released year: 2017
IMDb rating: 7.4
Age rating: R
Genres: Drama, Romance
Runtime: 2h 10min
Director: Paul Thomas Anderson

Title: Rainbow
Released year: 2022
IMDb rating: 4.5
Age rating: R
Genres: Fantasy, Drama, Comedy, Music & Musical
Runtime: 1h 57min
Director: Paco León

Title: Sebastian Maniscalco: Is it Me?
Released year: 2022
IMDb rating: 6.4
Age rating: R
Genres: Comedy, Reality TV, Documentary
Runtime: 1h 0min
Director: Peter Segal

Title: Disappearance at Clifton Hill
Released year: 2020
IMDb rating: 5.4
Age rating: N/A
Genres: Mystery & Thriller, Drama
Runtime: 1h 40min
Director: Albert Shin

Title: The Soccer Football Movie
Released year: 2022
IMDb rating: 3.1
Age rating: PG
Genres: Animation, Comedy, Kids & Family, Sport
Runtime: 1h 14min
Director: Mitch Schauer

Title: Won't You Be My Neighbor?
Released year: 2018
IMDb rating: 8.4
Age rating: PG-13
G

Title: Straight Up
Released year: 2020
IMDb rating: 6.9
Age rating: N/A
Genres: Romance, Comedy
Runtime: 1h 35min
Director: James Sweeney

Title: The Post-Truth World
Released year: 2022
IMDb rating: 6.5
Age rating: N/A
Genres: Mystery & Thriller, Crime, Drama
Runtime: 2h 0min
Director: If Chen

Title: My Next Guest with David Letterman and Volodymyr Zelenskyy
Released year: 2022
IMDb rating: N/A
Age rating: N/A
Genres: Documentary
Runtime: 44min
Director: Michael Steed

Title: Private Lesson
Released year: 2022
IMDb rating: 4.5
Age rating: R
Genres: Romance, Comedy
Runtime: 1h 29min
Director: Kıvanç Baruönü

Title: Britney vs. Spears
Released year: 2021
IMDb rating: 6.4
Age rating: N/A
Genres: Documentary, Music & Musical
Runtime: 1h 33min
Director: Erin Lee Carr

Title: Southpaw
Released year: 2015
IMDb rating: 7.3
Age rating: R
Genres: Action & Adventure, Drama, Crime, Sport
Runtime: 2h 3min
Director: Antoine Fuqua

Title: Blood Line
Released year: 2020
IMDb rating: 2.4
Age rating: 

...

IMDb rating: 6.9
Age rating: PG-13
Genres: Animation
Runtime: 8min
Director: Timothy Ware-Hill

Title: The Amazing Praybeyt Benjamin
Released year: 2014
IMDb rating: 4.1
Age rating: N/A
Genres: Drama, Comedy, Action & Adventure, Fantasy, Romance
Runtime: 1h 45min
Director: Wenn V. Deramas

Title: Double XL
Released year: 2022
IMDb rating: 4.6
Age rating: N/A
Genres: Drama, Comedy
Runtime: 2h 8min
Director: Satramm Ramani

Title: Weapon of Choice
Released year: 2018
IMDb rating: 4.9
Age rating: N/A
Genres: Documentary, Made in Europe
Runtime: 1h 30min
Director: Fritz Ofner

Title: Ricardo Quevedo: los amargados somos más
Released year: 2019
IMDb rating: 7.2
Age rating: N/A
Genres: Comedy
Runtime: 58min
Director: Fernando Ayllón

Title: A Babysitter's Guide to Monster Hunting
Released year: 2020
IMDb rating: 5.4
Age rating: PG
Genres: Comedy, Fantasy, Kids & Family
Runtime: 1h 34min
Director: Rachel Talalay

Title: Hamid
Released year: 2019
IMDb rating: 7.7
Age rating: N/A
Genres: Drama


IMDb rating: 3.9
Age rating: N/A
Genres: Drama, Mystery & Thriller, History
Runtime: 2h 8min
Director: Kunle Afolayan

Title: Life Overtakes Me
Released year: 2019
IMDb rating: 5.0
Age rating: N/A
Genres: Documentary
Runtime: 39min
Director: Kristine Samuelson

Title: The Irishman: In Conversation
Released year: 2019
IMDb rating: N/A
Age rating: N/A
Genres: Documentary
Runtime: 23min
Director: Kristine Samuelson

Title: Feo pero Sabroso
Released year: 2019
IMDb rating: 4.3
Age rating: N/A
Genres: Comedy
Runtime: 1h 30min
Director: Fernando Ayllón

Title: The American Meme
Released year: 2018
IMDb rating: 6.2
Age rating: N/A
Genres: Documentary
Runtime: 1h 35min
Director: Bert Marcus

Title: Happy Old Year
Released year: 2019
IMDb rating: 7.2
Age rating: N/A
Genres: Drama, Romance
Runtime: 1h 53min
Director: Nawapol Thamrongrattanarit

Title: Cobalt Blue
Released year: 2022
IMDb rating: 6.5
Age rating: N/A
Genres: Drama, Romance
Runtime: 1h 52min
Director: Sachin Kundalkar

Title: D@bbe

IMDb rating: 4.5
Age rating: R
Genres: Horror
Runtime: 1h 27min
Director: Justin P. Lange

Title: Lift Like a Girl
Released year: 2020
IMDb rating: 7.7
Age rating: N/A
Genres: Documentary
Runtime: 1h 33min
Director: Mayye Zayed

Title: Thunder Force
Released year: 2021
IMDb rating: 4.5
Age rating: PG-13
Genres: Comedy, Science-Fiction, Action & Adventure, Fantasy
Runtime: 1h 45min
Director: Ben Falcone

Title: Close
Released year: 2019
IMDb rating: 5.7
Age rating: R
Genres: Mystery & Thriller, Action & Adventure, Drama
Runtime: 1h 34min
Director: Vicky Jewson

Title: Ip Man 4: The Finale
Released year: 2019
IMDb rating: 7.0
Age rating: PG
Genres: Drama, History, Action & Adventure
Runtime: 1h 45min
Director: Wilson Yip

Title: Crouching Tiger, Hidden Dragon: Sword of Destiny
Released year: 2016
IMDb rating: 6.1
Age rating: PG-13
Genres: Drama, Fantasy, Action & Adventure
Runtime: 1h 36min
Director: Yuen Woo-ping

Title: The Informer
Released year: 2019
IMDb rating: 6.6
Age rating: R
Ge

Title: Unicorn Store
Released year: 2017
IMDb rating: 5.6
Age rating: N/A
Genres: Drama, Comedy, Fantasy
Runtime: 1h 32min
Director: Brie Larson

Title: Rose Island
Released year: 2020
IMDb rating: 7.0
Age rating: N/A
Genres: Comedy, Drama
Runtime: 1h 57min
Director: Sydney Sibilia

Title: Kumari
Released year: 2022
IMDb rating: 6.4
Age rating: N/A
Genres: Horror, Mystery & Thriller
Runtime: 2h 17min
Director: Sydney Sibilia

Title: Schumacher
Released year: 2021
IMDb rating: 7.4
Age rating: N/A
Genres: Sport, Documentary
Runtime: 1h 52min
Director: Hanns-Bruno Kammertöns

Title: Lady J
Released year: 2018
IMDb rating: 6.8
Age rating: N/A
Genres: Drama, Romance, Made in Europe
Runtime: 1h 49min
Director: Emmanuel Mouret

Title: On Body and Soul
Released year: 2017
IMDb rating: 7.5
Age rating: N/A
Genres: Drama, Fantasy, Romance
Runtime: 1h 56min
Director: Ildikó Enyedi

Title: Private Life
Released year: 2018
IMDb rating: 7.2
Age rating: R
Genres: Drama, Comedy
Runtime: 2h 3min
Directo

Genres: Comedy, Drama
Runtime: 1h 42min
Director: Norihisa Hiranuma

Title: Padavettu
Released year: 2022
IMDb rating: 7.0
Age rating: N/A
Genres: Comedy, Action & Adventure, Drama, Mystery & Thriller
Runtime: 2h 25min
Director: Liju Krishna

Title: That Thing Called Tadhana
Released year: 2014
IMDb rating: 7.4
Age rating: N/A
Genres: Romance, Comedy, Drama
Runtime: 1h 50min
Director: Antoinette Jadaone

Title: Dolly Kitty and Those Twinkling Stars
Released year: 2019
IMDb rating: 5.2
Age rating: R
Genres: Drama, Comedy
Runtime: 2h 0min
Director: Alankrita Shrivastava

Title: My Own Man
Released year: 2014
IMDb rating: 6.4
Age rating: N/A
Genres: Comedy, Drama, Kids & Family, Documentary
Runtime: 1h 22min
Director: David Sampliner

Title: The Road to El Camino: Behind the Scenes of El Camino: A Breaking Bad Movie
Released year: 2019
IMDb rating: 7.1
Age rating: PG-13
Genres: Documentary
Runtime: 13min
Director: David Sampliner

Title: Saheb Biwi Aur Gangster Returns
Released year: 2013

In [8]:
# Return a dictionary with all the movie data

movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
Netflix_df = pd.DataFrame(movie_data)

NameError: name 'Movies' is not defined

In [17]:
Netflix_df

,title,year,genre,runtime,rating,age_rating,director_name
0,A Man Called Otto,2022,"Comedy, Drama",2h 6min,7.4,PG-13,Marc Forster
1,Soft & Quiet,2022,"Drama, Mystery & Thriller, Horror",1h 31min,6.3,R,Beth de Araújo
2,Above Suspicion,2019,"Mystery & Thriller, Crime, Action & Adventure",1h 44min,5.7,R,Phillip Noyce
3,Bullet Train,2022,"Mystery & Thriller, Comedy, Action & Adventure",2h 6min,7.3,R,David Leitch
4,The Glass Castle,2017,Drama,2h 7min,7.1,PG-13,Destin Daniel Cretton
...,...,...,...,...,...,...,...
1953,The Ridiculous 6,2015,"Western, Action & Adventure, Comedy",1h 59min,4.8,PG-13,Frank Coraci
1954,Soa,2020,"Documentary, Drama",1h 11min,6.3,N/A,Raquel Castro
1955,Pan,2015,"Fantasy, Action & Adventure, Comedy, Kids & Fa...",1h 51min,5.7,PG,Joe Wright
1956,Fast & Feel Love,2022,"Romance, Action & Adventure, Comedy, Sport, My...",2h 12min,6.8,N/A,Nawapol Thamrongrattanarit


In [ ]:
# save the combined DataFrame as a CSV file
Netflix_df.to_csv('Netflix_df.csv', index=False)

 ## Getting all AmazonPrime Movies Data From 2013-2022.

In [21]:
def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/amazon-prime-video/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

    # Get the IMDb rating
    def get_imdb_rating(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
            rating = imdb_tag.text.strip().split()[0]
        except AttributeError:
            rating = 'N/A'
        return rating
    rating = get_imdb_rating(url)
    ratings_list.append(rating)
    print(f"IMDb rating: {rating}")

    # Get the TV age rating
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    if age_rating_div:
        age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    else:
        age_rating = 'N/A'
    Age_Rating.append(age_rating)
    print(f"Age rating: {age_rating}")
    
    #get the movie genres
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    try:
        genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    except AttributeError:
        genre = 'N/A'
    genres.append(genre)
    print(f'Genres: {genre}')

    # Get the movie runtime
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    runtimes.append(runtime)
    print(f"Runtime: {runtime}")

    # Get the movie director
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    if director_div:
        director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
        directors.append(director_name)
    else:
        directors.append('N/A')
    print(f"Director: {director_name}\n")



Title: Ghosted
Released year: 2023
IMDb rating: N/A
Age rating: PG-13
Genres: Action & Adventure, Comedy, Romance
Runtime: 2h 0min
Director: Dexter Fletcher

Title: The Program
Released year: 2015
IMDb rating: 6.5
Age rating: PG-13
Genres: Drama, Sport, Made in Europe
Runtime: 1h 44min
Director: Stephen Frears

Title: Army of One
Released year: 2016
IMDb rating: 5.2
Age rating: R
Genres: Comedy, Crime
Runtime: 1h 32min
Director: Larry Charles

Title: Coherence
Released year: 2013
IMDb rating: 7.2
Age rating: PG
Genres: Mystery & Thriller, Horror, Drama, Science-Fiction
Runtime: 1h 29min
Director: James Ward Byrkit

Title: Gordon Lightfoot: If You Could Read My Mind
Released year: 2019
IMDb rating: 7.7
Age rating: N/A
Genres: Documentary, Music & Musical
Runtime: 1h 28min
Director: Martha Kehoe

Title: Nope
Released year: 2022
IMDb rating: 6.8
Age rating: R
Genres: Horror, Science-Fiction, Mystery & Thriller, Fantasy
Runtime: 2h 10min
Director: Jordan Peele

Title: Top Gun: Maverick
Rel

Title: Run to Me
Released year: 2016
IMDb rating: 5.1
Age rating: N/A
Genres: Mystery & Thriller, Drama, Sport
Runtime: 1h 40min
Director: Philippe Gagnon

Title: Kill Me Three Times
Released year: 2015
IMDb rating: 5.9
Age rating: R
Genres: Mystery & Thriller, Comedy, Action & Adventure, Crime
Runtime: 1h 30min
Director: Kriv Stenders

Title: Life Itself
Released year: 2014
IMDb rating: 7.8
Age rating: R
Genres: Documentary
Runtime: 2h 1min
Director: Steve James

Title: Sonic the Hedgehog 2
Released year: 2022
IMDb rating: 6.5
Age rating: PG
Genres: Comedy, Kids & Family, Fantasy, Science-Fiction, Animation, Action & Adventure
Runtime: 2h 2min
Director: Jeff Fowler

Title: Ambulance
Released year: 2022
IMDb rating: 6.1
Age rating: R
Genres: Mystery & Thriller, Crime, Drama, Horror, Action & Adventure
Runtime: 2h 16min
Director: Michael Bay

Title: Tag
Released year: 2018
IMDb rating: 6.5
Age rating: R
Genres: Action & Adventure, Comedy
Runtime: 1h 41min
Director: Jeff Tomsic

Title: W

Title: Night Train to Lisbon
Released year: 2013
IMDb rating: 6.8
Age rating: R
Genres: Mystery & Thriller, Romance, Made in Europe
Runtime: 1h 51min
Director: Bille August

Title: Abacus: Small Enough to Jail
Released year: 2017
IMDb rating: 7.1
Age rating: N/A
Genres: Documentary
Runtime: 1h 28min
Director: Steve James

Title: Pink
Released year: 2018
IMDb rating: N/A
Age rating: N/A
Genres: Drama
Runtime: 1h 20min
Director: Austin Herring

Title: Identity Thief
Released year: 2013
IMDb rating: 5.7
Age rating: R
Genres: Comedy, Crime, Drama
Runtime: 1h 51min
Director: Seth Gordon

Title: A Comedy of Horrors: Volume 1
Released year: 2021
IMDb rating: N/A
Age rating: R
Genres: Horror
Runtime: 1h 35min
Director: Matt Servitto

Title: 'Til Death Do Us Part
Released year: 2017
IMDb rating: 4.6
Age rating: PG-13
Genres: Mystery & Thriller, Drama, Comedy, Romance, Made in Europe
Runtime: 1h 39min
Director: Chris Stokes

Title: Valerian and the City of a Thousand Planets
Released year: 2017


Title: Beautiful Boy
Released year: 2018
IMDb rating: 7.3
Age rating: R
Genres: Drama
Runtime: 2h 0min
Director: Felix van Groeningen

Title: Mysterious Circumstance: The Death of Meriwether Lewis
Released year: 2022
IMDb rating: 4.8
Age rating: N/A
Genres: Western, Mystery & Thriller
Runtime: 1h 55min
Director: Clark Richey

Title: Strong Enough
Released year: 2022
IMDb rating: 6.2
Age rating: N/A
Genres: Drama, Sport
Runtime: 1h 10min
Director: John Foss

Title: The Debt Collector
Released year: 2018
IMDb rating: 5.8
Age rating: N/A
Genres: Action & Adventure, Comedy
Runtime: 1h 36min
Director: Jesse V. Johnson

Title: Red River Road
Released year: 2021
IMDb rating: 5.6
Age rating: N/A
Genres: Mystery & Thriller, Horror
Runtime: 1h 30min
Director: Paul Schuyler

Title: Status Update
Released year: 2018
IMDb rating: 5.8
Age rating: PG-13
Genres: Fantasy, Science-Fiction, Comedy, Romance
Runtime: 1h 46min
Director: Scott Speer

Title: Transformers: Age of Extinction
Released year: 2014

Genres: Drama, History, Romance
Runtime: 1h 49min
Director: Susanne Bier

Title: Nicole, her Ex & the Killer
Released year: 2022
IMDb rating: 6.6
Age rating: N/A
Genres: Comedy, Horror
Runtime: 1h 30min
Director: Bob Akins

Title: The Manson Brothers Midnight Zombie Massacre
Released year: 2021
IMDb rating: 4.4
Age rating: R
Genres: Horror, Comedy
Runtime: 1h 32min
Director: Max Martini

Title: The House on Mansfield Street
Released year: 2018
IMDb rating: 3.8
Age rating: PG-13
Genres: Horror
Runtime: 1h 14min
Director: Richard Mansfield

Title: 12 Feet Deep
Released year: 2017
IMDb rating: 5.2
Age rating: N/A
Genres: Horror, Mystery & Thriller
Runtime: 1h 26min
Director: Matt Eskandari

Title: Blush
Released year: 2019
IMDb rating: 4.9
Age rating: N/A
Genres: Drama
Runtime: 1h 41min
Director: Debra Eisenstadt

Title: Lancaster
Released year: 2022
IMDb rating: 7.8
Age rating: N/A
Genres: Documentary
Runtime: 1h 50min
Director: Ant Palmer

Title: Cyrano
Released year: 2022
IMDb rating: 

...

Genres: Horror, Mystery & Thriller, Drama
Runtime: 1h 13min
Director: Felix Bartke

Title: Touch Chesi Chudu
Released year: 2018
IMDb rating: 4.7
Age rating: G
Genres: Action & Adventure, Crime, Comedy
Runtime: 2h 0min
Director: Vikram Sirikonda

Title: Vellai Pookal
Released year: 2019
IMDb rating: 7.1
Age rating: PG
Genres: Drama, Mystery & Thriller, Crime
Runtime: 2h 10min
Director: Vivek Elangovan

Title: Time Is Up
Released year: 2021
IMDb rating: 4.2
Age rating: N/A
Genres: Romance, Drama
Runtime: 1h 48min
Director: Elisa Amoruso

Title: Catching the Black Widow
Released year: 2017
IMDb rating: 6.2
Age rating: N/A
Genres: Drama, Crime
Runtime: 1h 32min
Director: Thomas Robins

Title: Final Stop
Released year: 2021
IMDb rating: N/A
Age rating: N/A
Genres: Mystery & Thriller
Runtime: 1h 28min
Director: Thomas Robins

Title: Becoming the G.O.A.T.: The Tom Brady Story
Released year: 2021
IMDb rating: 4.3
Age rating: N/A
Genres: Documentary
Runtime: 1h 7min
Director: Johannes Guttenku

Title: Yuvarathnaa
Released year: 2021
IMDb rating: 6.2
Age rating: N/A
Genres: Action & Adventure, Drama
Runtime: 2h 41min
Director: Santhosh Ananddram

Title: Teacher of the Year
Released year: 2015
IMDb rating: 5.3
Age rating: N/A
Genres: Comedy, Drama
Runtime: 1h 22min
Director: Jason Strouse

Title: A Fine Step
Released year: 2014
IMDb rating: 5.3
Age rating: PG
Genres: Drama
Runtime: 1h 30min
Director: Jonathan Meyers

Title: Dark Intentions
Released year: 2015
IMDb rating: 5.3
Age rating: N/A
Genres: Drama, Horror, Mystery & Thriller
Runtime: 1h 30min
Director: Chris Sivertson

Title: 11:55
Released year: 2017
IMDb rating: 5.1
Age rating: N/A
Genres: Drama, Horror, Crime
Runtime: 1h 20min
Director: Oh In-chun

Title: One Cut Two Cut
Released year: 2022
IMDb rating: 4.7
Age rating: N/A
Genres: Comedy
Runtime: 1h 28min
Director: Vamsidhar Bhogaraju

Title: Nota
Released year: 2018
IMDb rating: 6.2
Age rating: N/A
Genres: Drama
Runtime: 2h 26min
Director: Anand Shankar

Title: Edie

Title: Two for Joy
Released year: 2018
IMDb rating: 6.2
Age rating: PG-13
Genres: Drama
Runtime: 1h 26min
Director: Tom Beard

Title: A Very Sordid Wedding
Released year: 2017
IMDb rating: 5.4
Age rating: N/A
Genres: Comedy
Runtime: 1h 49min
Director: Del Shores

Title: The Priest
Released year: 2021
IMDb rating: 6.0
Age rating: N/A
Genres: Drama, Mystery & Thriller, Horror
Runtime: 2h 20min
Director: Jofin T. Chacko

Title: I Hate the Man in My Basement
Released year: 2020
IMDb rating: 5.8
Age rating: R
Genres: Comedy, Drama, Crime, Romance, Mystery & Thriller
Runtime: 1h 43min
Director: Dustin Cook

Title: Dirty God
Released year: 2019
IMDb rating: 6.5
Age rating: N/A
Genres: Drama, Made in Europe
Runtime: 1h 44min
Director: Sacha Polak

Title: Garden of Hedon
Released year: 2013
IMDb rating: 3.2
Age rating: N/A
Genres: Horror
Runtime: 1h 59min
Director: Kevin Kangas

Title: Frank and Ava
Released year: 2020
IMDb rating: 3.6
Age rating: N/A
Genres: Drama, History
Runtime: 1h 50min
Di

IMDb rating: 5.2
Age rating: N/A
Genres: Comedy
Runtime: 1h 2min
Director: Alex Ferrari

Title: Veerappan
Released year: 2016
IMDb rating: 5.3
Age rating: PG-13
Genres: Action & Adventure, Crime
Runtime: 2h 5min
Director: Ram Gopal Varma

Title: Grand Cru
Released year: 2018
IMDb rating: 6.2
Age rating: N/A
Genres: Documentary
Runtime: 1h 20min
Director: Ram Gopal Varma

Title: Thrill Ride
Released year: 2017
IMDb rating: 5.4
Age rating: PG
Genres: Comedy, Action & Adventure, Fantasy
Runtime: 1h 26min
Director: Chris Parrish

Title: 7500
Released year: 2019
IMDb rating: 6.3
Age rating: R
Genres: Drama, Mystery & Thriller, Action & Adventure
Runtime: 1h 33min
Director: Patrick Vollrath

Title: Out on Stage
Released year: 2018
IMDb rating: 7.1
Age rating: N/A
Genres: Comedy
Runtime: 1h 22min
Director: Samuel Brownfield

Title: The Final Wish
Released year: 2019
IMDb rating: 4.3
Age rating: R
Genres: Horror
Runtime: 1h 35min
Director: Timothy Woodward Jr.

Title: Mary and The Witch's Flow

Title: The Fighting Preacher
Released year: 2019
IMDb rating: 5.8
Age rating: PG
Genres: History, Comedy, Drama
Runtime: 1h 40min
Director: T.C. Christensen

Title: Sideshow
Released year: 2022
IMDb rating: 5.5
Age rating: N/A
Genres: Comedy, Crime
Runtime: 1h 34min
Director: Adam Oldroyd

Title: My Dad Is Scrooge
Released year: 2014
IMDb rating: 4.4
Age rating: N/A
Genres: Comedy, Kids & Family
Runtime: 1h 27min
Director: Justin G. Dyck

Title: Varsity Punks
Released year: 2017
IMDb rating: 6.3
Age rating: N/A
Genres: Drama, Comedy, Sport
Runtime: 1h 40min
Director: Anthony Solorzano

Title: Eddie Griffin: Undeniable
Released year: 2018
IMDb rating: 5.1
Age rating: PG
Genres: Comedy
Runtime: 1h 14min
Director: Gobi M. Rahimi

Title: An American in Texas
Released year: 2017
IMDb rating: 5.1
Age rating: N/A
Genres: Drama, Crime, Music & Musical, War & Military, Made in Europe
Runtime: 1h 37min
Director: Anthony Pedone

Title: TQM
Released year: 2022
IMDb rating: 4.1
Age rating: N/A
Genr

In [9]:
# Return a dictionary with all the movie data
movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
Amazonprime_df = pd.DataFrame(movie_data)

NameError: name 'Movies' is not defined

In [10]:
Amazonprime_df

NameError: name 'Amazonprime_df' is not defined

In [4]:
Amazonprime_df.to_csv('Amazonprime.csv', index = False)

## Getting all Disney+ Movies from 2013-2022

In [31]:
def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()

    # Send a request to the website
    url = 'https://www.justwatch.com/us/provider/disney-plus/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

    # Get the IMDb rating
    def get_imdb_rating(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
            rating = imdb_tag.text.strip().split()[0]
        except AttributeError:
            rating = 'N/A'
        return rating
    rating = get_imdb_rating(url)
    ratings_list.append(rating)
    print(f"IMDb rating: {rating}")

    # Get the TV age rating
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    if age_rating_div:
        age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    else:
        age_rating = 'N/A'
    Age_Rating.append(age_rating)
    print(f"Age rating: {age_rating}")
    
    #get the movie genres
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    try:
        genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    except AttributeError:
        genre = 'N/A'
    genres.append(genre)
    print(f'Genres: {genre}')

    # Get the movie runtime
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    runtimes.append(runtime)
    print(f"Runtime: {runtime}")

    # Get the movie director
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    if director_div:
        director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
        directors.append(director_name)
    else:
        directors.append('N/A')
    print(f"Director: {director_name}\n")



Title: Ghosted
Released year: 2023
IMDb rating: N/A
Age rating: PG-13
Genres: Action & Adventure, Romance, Comedy
Runtime: 2h 0min
Director: Dexter Fletcher

Title: Guardians of the Galaxy
Released year: 2014
IMDb rating: 8.0
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure, Comedy
Runtime: 2h 1min
Director: James Gunn

Title: Guardians of the Galaxy Vol. 2
Released year: 2017
IMDb rating: 7.6
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure, Comedy
Runtime: 2h 17min
Director: James Gunn

Title: Black Panther: Wakanda Forever
Released year: 2022
IMDb rating: 6.7
Age rating: PG-13
Genres: Action & Adventure, Science-Fiction, Mystery & Thriller, Fantasy, Drama
Runtime: 2h 42min
Director: Ryan Coogler

Title: Free Guy
Released year: 2021
IMDb rating: 7.1
Age rating: PG-13
Genres: Action & Adventure, Science-Fiction, Comedy, Fantasy
Runtime: 1h 55min
Director: Shawn Levy

Title: The Guardians of the Galaxy Holiday Special
Released year: 2022
IMDb rating: 6.9
Age r

Title: Percy Jackson: Sea of Monsters
Released year: 2013
IMDb rating: 5.7
Age rating: PG
Genres: Action & Adventure, Kids & Family, Fantasy, Mystery & Thriller
Runtime: 1h 46min
Director: Thor Freudenthal

Title: Captain Marvel
Released year: 2019
IMDb rating: 6.8
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure
Runtime: 2h 4min
Director: Anna Boden

Title: The Rescue
Released year: 2021
IMDb rating: 8.4
Age rating: PG
Genres: Mystery & Thriller, Drama, Documentary, Action & Adventure
Runtime: 1h 47min
Director: Jimmy Chin

Title: Remembering
Released year: 2022
IMDb rating: 5.5
Age rating: G
Genres: Fantasy, Drama
Runtime: 9min
Director: Elijah Allan-Blitz

Title: Turning Red
Released year: 2022
IMDb rating: 7.0
Age rating: PG
Genres: Comedy, Fantasy, Animation, Kids & Family
Runtime: 1h 40min
Director: Domee Shi

Title: Soul
Released year: 2020
IMDb rating: 8.0
Age rating: PG
Genres: Kids & Family, Comedy, Fantasy, Drama, Animation, Music & Musical, Science-Fiction
Runt

IMDb rating: 7.8
Age rating: PG-13
Genres: Fantasy, Action & Adventure, Science-Fiction
Runtime: 2h 15min
Director: J.J. Abrams

Title: Captain America: The Winter Soldier
Released year: 2014
IMDb rating: 7.8
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure, Mystery & Thriller
Runtime: 2h 8min
Director: Anthony Russo

Title: Free Solo
Released year: 2018
IMDb rating: 8.1
Age rating: PG-13
Genres: Documentary, Action & Adventure, Sport
Runtime: 1h 40min
Director: Jimmy Chin

Title: Retrograde
Released year: 2022
IMDb rating: 7.4
Age rating: R
Genres: War & Military, Documentary, History
Runtime: 1h 34min
Director: Matthew Heineman

Title: Incredibles 2
Released year: 2018
IMDb rating: 7.6
Age rating: PG
Genres: Action & Adventure, Animation, Comedy, Kids & Family, Science-Fiction
Runtime: 1h 58min
Director: Brad Bird

Title: LEGO Star Wars Summer Vacation
Released year: 2022
IMDb rating: 5.6
Age rating: G
Genres: Animation, Science-Fiction, Comedy, Kids & Family, Fantasy, A

IMDb rating: 6.0
Age rating: PG
Genres: Kids & Family, Mystery & Thriller, Comedy, Fantasy, Drama
Runtime: 1h 39min
Director: Tom McCarthy

Title: Captain America: Civil War
Released year: 2016
IMDb rating: 7.8
Age rating: PG-13
Genres: Action & Adventure, Science-Fiction
Runtime: 2h 29min
Director: Anthony Russo

Title: Descendants 3
Released year: 2019
IMDb rating: 6.4
Age rating: PG
Genres: Music & Musical, Action & Adventure, Fantasy, Kids & Family, Mystery & Thriller, Drama, Reality TV
Runtime: 1h 46min
Director: Kenny Ortega

Title: Alexander and the Terrible, Horrible, No Good, Very Bad Day
Released year: 2014
IMDb rating: 6.2
Age rating: PG
Genres: Comedy, Drama, Kids & Family, Fantasy, Made in Europe
Runtime: 1h 15min
Director: Miguel Arteta

Title: X-Men: Days of Future Past
Released year: 2014
IMDb rating: 7.9
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure, Fantasy, Mystery & Thriller
Runtime: 2h 12min
Director: Bryan Singer

Title: Howard
Released year: 2018


Title: Explorer: The Last Tepui
Released year: 2022
IMDb rating: 6.9
Age rating: N/A
Genres: Documentary
Runtime: 1h 0min
Director: Renan Ozturk

Title: Aladdin
Released year: 2019
IMDb rating: 6.9
Age rating: PG
Genres: Fantasy, Action & Adventure, Romance, Comedy, Kids & Family, Drama
Runtime: 2h 8min
Director: Guy Ritchie

Title: Burrow
Released year: 2020
IMDb rating: N/A
Age rating: G
Genres: Animation
Runtime: 6min
Director: Madeline Sharafian

Title: Solo: A Star Wars Story
Released year: 2018
IMDb rating: 6.9
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure, Fantasy
Runtime: 2h 15min
Director: Ron Howard

Title: From the Ashes
Released year: 2017
IMDb rating: 6.9
Age rating: PG
Genres: Documentary, Sport, History, Action & Adventure, Science-Fiction
Runtime: 1h 22min
Director: Michael Bonfiglio

Title: Toy Story 4
Released year: 2019
IMDb rating: 7.7
Age rating: G
Genres: Kids & Family, Action & Adventure, Animation, Comedy, Fantasy, Documentary
Runtime: 1h 40min
D

...

IMDb rating: N/A
Age rating: G
Genres: Comedy
Runtime: 2h 10min
Director: Robert Vince

Title: The Swap
Released year: 2016
IMDb rating: 6.0
Age rating: PG
Genres: Comedy, Fantasy, Kids & Family
Runtime: 1h 29min
Director: Jay Karas

Title: Auntie Edna
Released year: 2018
IMDb rating: N/A
Age rating: N/A
Genres: Comedy
Runtime: 5min
Director: Ted Mathot

Title: The Little Mermaid Live!
Released year: 2019
IMDb rating: 5.6
Age rating: N/A
Genres: Kids & Family, Fantasy, Romance, Animation, Comedy, Music & Musical
Runtime: 1h 23min
Director: Hamish Hamilton

Title: Inner Workings
Released year: 2016
IMDb rating: N/A
Age rating: G
Genres: Animation
Runtime: 6min
Director: Leo Matsuda

Title: The Wonderful Spring of Mickey Mouse
Released year: 2022
IMDb rating: N/A
Age rating: G
Genres: Animation, Comedy, Action & Adventure
Runtime: 22min
Director: Karl Hadrika

Title: The Wizards Return: Alex vs. Alex
Released year: 2013
IMDb rating: 6.3
Age rating: PG
Genres: Comedy, Action & Adventure, 

Title: Jane Goodall: The Hope
Released year: 2020
IMDb rating: 7.9
Age rating: N/A
Genres: Documentary, Action & Adventure
Runtime: 1h 28min
Director: Kim Woodard

Title: Forky Asks a Question: What Is Love?
Released year: 2019
IMDb rating: N/A
Age rating: N/A
Genres: Animation
Runtime: 4min
Director: Bob Peterson

Title: Mega Hammerhead
Released year: 2016
IMDb rating: 6.5
Age rating: N/A
Genres: Documentary
Runtime: 1h 0min
Director: Brandon Hill

Title: The Real Right Stuff
Released year: 2020
IMDb rating: 7.2
Age rating: PG
Genres: History, Documentary
Runtime: 1h 30min
Director: Tom Jennings

Title: Forky Asks A Question: What Is Money?
Released year: 2019
IMDb rating: N/A
Age rating: N/A
Genres: Animation, Comedy
Runtime: 3min
Director: Bob Peterson

Title: Feliz NaviDAD
Released year: 2020
IMDb rating: N/A
Age rating: N/A
Genres: Romance
Runtime: 1h 20min
Director: Melissa Joan Hart

Title: High School Musical: The Musical: The Series: The Special
Released year: 2019
IMDb rating

IMDb rating: 7.1
Age rating: G
Genres: Documentary
Runtime: 1h 17min
Director: Keith Scholey

Title: PRIDE: To Be Seen - A Soul of a Nation Presentation
Released year: 2022
IMDb rating: 6.4
Age rating: N/A
Genres: Documentary
Runtime: 41min
Director: Tine

Title: Built for Mars: The Perseverance Rover
Released year: 2021
IMDb rating: 6.9
Age rating: PG-13
Genres: Documentary
Runtime: 1h 28min
Director: Mark Davis

Title: Flooded Tombs of the Nile
Released year: 2021
IMDb rating: 6.3
Age rating: PG
Genres: Documentary, History
Runtime: 44min
Director: Katie Bauer Murdock

Title: Game of Sharks
Released year: 2022
IMDb rating: 5.2
Age rating: PG
Genres: Documentary
Runtime: 44min
Director: Anna Derryberry

Title: The Good, The Bad, The Hungry
Released year: 2019
IMDb rating: 6.8
Age rating: N/A
Genres: Documentary, Sport
Runtime: 1h 17min
Director: Nicole Lucas Haimes

Title: The Lost City Of Machu Picchu
Released year: 2019
IMDb rating: 6.9
Age rating: PG-13
Genres: History, Documentary

IMDb rating: N/A
Age rating: N/A
Genres: Documentary
Runtime: 1h 19min
Director: Ken Rodgers

Title: Epic Holiday Showdown
Released year: 2020
IMDb rating: 4.7
Age rating: N/A
Genres: 
Runtime: 49min
Director: Ken Rodgers

Title: Dinner Is Served
Released year: 2021
IMDb rating: N/A
Age rating: N/A
Genres: Drama
Runtime: 21min
Director: Hao Zheng

Title: Mission Pluto
Released year: 2015
IMDb rating: N/A
Age rating: N/A
Genres: Documentary
Runtime: 44min
Director: Nic Young

Title: Shark Side of the Moon
Released year: 2022
IMDb rating: N/A
Age rating: PG
Genres: 
Runtime: 43min
Director: Sarah V. Burns

Title: Baby Sharks
Released year: 2022
IMDb rating: 6.3
Age rating: PG
Genres: Documentary
Runtime: 45min
Director: Adam Geiger

Title: Doc & Darryl
Released year: 2016
IMDb rating: N/A
Age rating: N/A
Genres: Documentary
Runtime: 1h 18min
Director: Michael Bonfiglio

Title: Man Among Cheetahs
Released year: 2017
IMDb rating: 6.8
Age rating: PG
Genres: Documentary
Runtime: 44min
Direct

Title: Petra: Secrets of the Ancient Builders
Released year: 2019
IMDb rating: N/A
Age rating: G
Genres: Documentary
Runtime: 44min
Director: Andy Seestedt

Title: Weekend Family Christmas Special
Released year: 2022
IMDb rating: N/A
Age rating: N/A
Genres: Comedy
Runtime: 46min
Director: Andy Seestedt

Title: It Was Always Me: Behind the Story
Released year: 2022
IMDb rating: 8.0
Age rating: N/A
Genres: Documentary
Runtime: 31min
Director: Marco Colantoni

Title: Sharks That Eat Everything
Released year: 2022
IMDb rating: N/A
Age rating: PG-13
Genres: Documentary
Runtime: 44min
Director: Marco Colantoni

Title: The Hidden Kingdoms of China
Released year: 2020
IMDb rating: 6.9
Age rating: PG
Genres: Documentary
Runtime: 1h 28min
Director: Ben Wallis

Title: Far From Raven's Home
Released year: 2021
IMDb rating: 4.8
Age rating: G
Genres: Animation, Comedy
Runtime: 11min
Director: Ben Wallis

Title: Spring Shorts-Tacular with the Ghost and Molly McGee
Released year: 2022
IMDb rating: N/A

In [32]:
# Return a dictionary with all the movie data
movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
disney_df = pd.DataFrame(movie_data)

In [33]:
disney_df

,title,year,genre,runtime,rating,age_rating,director_name
0,Ghosted,2023,"Action & Adventure, Romance, Comedy",2h 0min,N/A,PG-13,Dexter Fletcher
1,Guardians of the Galaxy,2014,"Science-Fiction, Action & Adventure, Comedy",2h 1min,8.0,PG-13,James Gunn
2,Guardians of the Galaxy Vol. 2,2017,"Science-Fiction, Action & Adventure, Comedy",2h 17min,7.6,PG-13,James Gunn
3,Black Panther: Wakanda Forever,2022,"Action & Adventure, Science-Fiction, Mystery &...",2h 42min,6.7,PG-13,Ryan Coogler
4,Free Guy,2021,"Action & Adventure, Science-Fiction, Comedy, F...",1h 55min,7.1,PG-13,Shawn Levy
...,...,...,...,...,...,...,...
596,Heroes of the Long Road Home with Martha Raddatz,2017,,46min,N/A,N/A,Pierre sérisier
597,Egypt's Treasure Guardians,2016,"Documentary, History",44min,6.1,N/A,N/A
598,My Music Story: Yoshiki,2020,"Documentary, Music & Musical",47min,7.3,N/A,N/A
599,Moana Sing-Along,2017,,1h 47min,N/A,N/A,N/A


In [34]:
disney_df.to_csv('disney.csv', index = False)

## Getting all Hulu movies from 2013-2022

In [12]:
def get_movies_data():
    # Set up Selenium
    driver = webdriver.Chrome()
    
    # Send a request to the website
    url =  'https://www.justwatch.com/us/provider/hulu/movies?release_year_from=2013&release_year_until=2022&sort_by=trending_7_day'
    driver.get(url)

    # Scroll down to load more movies
    scroll_down(driver)

    # Get the HTML content of the website
    html = driver.page_source

    # Parse the HTML using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')

    # Extract the URLs of all movies
    urls = []
    imgs = soup.find_all('img', {'data-v-630d1f08': '', 'class': 'picture-comp__img'})
    for img in imgs:
        if 'alt' in img.attrs:
            url = img.find_parent('a')['href']
            urls.append('https://www.justwatch.com' + url)

    # Return the list of URLs
    return urls

# Create empty lists for movie data
urls = get_movies_data()
Movies = []
Released_year = []
ratings_list = []
genres = []
Age_Rating = []
runtimes = []
directors = []

for url in urls:
    # Get the movie title
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    title_block = soup.find('div', {'class': 'title-block'})
    title = title_block.find('h1').text.strip()
    Movies.append(title)
    print(f"Title: {title}")

    # Get the released year
    year_div = soup.find('div', {'class': 'title-block'}).find('span', {'class': 'text-muted'})
    year = year_div.text.strip()[1:5] if year_div else 'N/A'
    Released_year.append(year)
    print(f"Released year: {year}")

    # Get the IMDb rating
    def get_imdb_rating(url):
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            imdb_tag = soup.find('a', {'class': '', 'href': True}).find('img', alt='IMDB').find_parent('a')
            rating = imdb_tag.text.strip().split()[0]
        except AttributeError:
            rating = 'N/A'
        return rating
    rating = get_imdb_rating(url)
    ratings_list.append(rating)
    print(f"IMDb rating: {rating}")

    # Get the TV age rating
    age_rating_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Age rating')
    if age_rating_div:
        age_rating = age_rating_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()
    else:
        age_rating = 'N/A'
    Age_Rating.append(age_rating)
    print(f"Age rating: {age_rating}")
    
    #get the movie genres
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    genre_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Genres')
    try:
        genre = genre_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    except AttributeError:
        genre = 'N/A'
    genres.append(genre)
    print(f'Genres: {genre}')

    # Get the movie runtime
    runtime_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Runtime')
    if runtime_div:
        runtime = runtime_div.find_next_sibling('div', {'class': 'detail-infos__value'}).text.strip()   
    else:
        runtime = 'N/A'    
    runtimes.append(runtime)
    print(f"Runtime: {runtime}")


    # Get the movie director
    director_div = soup.find('div', {'class': 'detail-infos__subheading'}, text='Director')
    if director_div:
        director_name = director_div.find_next_sibling('div', {'class': 'detail-infos__value'}).find('span').find('a', {'class': 'title-credit-name'}).text.strip()
        directors.append(director_name)
    else:
        directors.append('N/A')
    print(f"Director: {director_name}\n")



Title: Ghosted
Released year: 2023
IMDb rating: N/A
Age rating: PG-13
Genres: Action & Adventure, Romance, Comedy
Runtime: 2h 0min
Director: Dexter Fletcher

Title: The Meg
Released year: 2018
IMDb rating: 5.6
Age rating: PG-13
Genres: Science-Fiction, Action & Adventure, Horror, Mystery & Thriller
Runtime: 1h 53min
Director: Jon Turteltaub

Title: Boss Level
Released year: 2021
IMDb rating: 6.8
Age rating: R
Genres: Mystery & Thriller, Science-Fiction, Action & Adventure, Comedy
Runtime: 1h 34min
Director: Joe Carnahan

Title: Escape Room
Released year: 2019
IMDb rating: 6.4
Age rating: PG-13
Genres: Horror, Mystery & Thriller, Drama, Action & Adventure, Science-Fiction
Runtime: 1h 40min
Director: Adam Robitel

Title: The Program
Released year: 2015
IMDb rating: 6.5
Age rating: PG-13
Genres: Drama, Sport, Made in Europe
Runtime: 1h 44min
Director: Stephen Frears

Title: Burn
Released year: 2019
IMDb rating: 5.7
Age rating: R
Genres: Mystery & Thriller, Crime, Comedy
Runtime: 1h 29min


Title: She Will
Released year: 2022
IMDb rating: 5.4
Age rating: N/A
Genres: Horror, Mystery & Thriller, Drama
Runtime: 1h 35min
Director: Charlotte Colbert

Title: Jumanji: Welcome to the Jungle
Released year: 2017
IMDb rating: 6.9
Age rating: PG-13
Genres: Action & Adventure, Comedy, Fantasy
Runtime: 1h 59min
Director: Jake Kasdan

Title: How to Train Your Dragon 2
Released year: 2014
IMDb rating: 7.8
Age rating: PG
Genres: Comedy, Fantasy, Animation, Action & Adventure, Kids & Family
Runtime: 1h 45min
Director: Dean DeBlois

Title: To the End
Released year: 2022
IMDb rating: 2.9
Age rating: R
Genres: Documentary
Runtime: 1h 43min
Director: Rachel Lears

Title: The Accursed
Released year: 2021
IMDb rating: 4.6
Age rating: N/A
Genres: Horror, Drama
Runtime: 1h 24min
Director: Elizabeta Vidovic

Title: Possessor Uncut
Released year: 2020
IMDb rating: 6.5
Age rating: R
Genres: Horror, Mystery & Thriller, Science-Fiction
Runtime: 1h 44min
Director: Brandon Cronenberg

Title: Fresh
Releas

IMDb rating: 4.2
Age rating: N/A
Genres: Mystery & Thriller, Action & Adventure
Runtime: 1h 38min
Director: Christian Sesma

Title: Flux Gourmet
Released year: 2022
IMDb rating: 5.9
Age rating: N/A
Genres: Drama, Horror, Comedy
Runtime: 1h 51min
Director: Peter Strickland

Title: Voyagers
Released year: 2021
IMDb rating: 5.4
Age rating: PG-13
Genres: Mystery & Thriller, Science-Fiction
Runtime: 1h 48min
Director: Neil Burger

Title: Lullaby
Released year: 2022
IMDb rating: 4.8
Age rating: N/A
Genres: Horror
Runtime: 1h 26min
Director: John R. Leonetti

Title: Just Getting Started
Released year: 2017
IMDb rating: 4.5
Age rating: PG-13
Genres: Comedy, Crime, Action & Adventure
Runtime: 1h 31min
Director: Ron Shelton

Title: Dinner in America
Released year: 2022
IMDb rating: 7.3
Age rating: N/A
Genres: Drama, Comedy, Music & Musical, Romance
Runtime: 1h 46min
Director: Adam Rehmeier

Title: Private Property
Released year: 2022
IMDb rating: 4.2
Age rating: R
Genres: Mystery & Thriller, Dra

Title: Mr. Jones
Released year: 2019
IMDb rating: 6.9
Age rating: N/A
Genres: Mystery & Thriller, Drama, Made in Europe
Runtime: 1h 58min
Director: Agnieszka Holland

Title: Transpecos
Released year: 2016
IMDb rating: 5.8
Age rating: N/A
Genres: Mystery & Thriller
Runtime: 1h 26min
Director: Greg Kwedar

Title: Dirty Weekend
Released year: 2015
IMDb rating: 5.0
Age rating: N/A
Genres: Drama, Comedy
Runtime: 1h 33min
Director: Neil LaBute

Title: Project Legion
Released year: 2022
IMDb rating: 2.1
Age rating: R
Genres: Horror, Mystery & Thriller, Science-Fiction
Runtime: 1h 29min
Director: Lance Kawas

Title: Offseason
Released year: 2022
IMDb rating: 4.9
Age rating: N/A
Genres: Horror, Science-Fiction, Mystery & Thriller, Drama, Fantasy
Runtime: 1h 23min
Director: Mickey Keating

Title: Philomena
Released year: 2013
IMDb rating: 7.6
Age rating: PG-13
Genres: Drama, Comedy, Made in Europe
Runtime: 1h 38min
Director: Stephen Frears

Title: This Mountain Life
Released year: 2018
IMDb rati

IMDb rating: 6.5
Age rating: N/A
Genres: Drama, Romance, Music & Musical
Runtime: 1h 26min
Director: Rashaad Ernesto Green

Title: A Long Way Down
Released year: 2014
IMDb rating: 6.3
Age rating: R
Genres: Drama, Comedy
Runtime: 1h 36min
Director: Pascal Chaumeil

Title: The Feels
Released year: 2017
IMDb rating: 5.1
Age rating: N/A
Genres: Comedy, Drama, Romance
Runtime: 1h 28min
Director: Jenée LaMarque

Title: Look at Me: XXXTentacion
Released year: 2022
IMDb rating: 7.5
Age rating: N/A
Genres: Documentary, Music & Musical
Runtime: 1h 48min
Director: Sabaah Folayan

Title: XX
Released year: 2017
IMDb rating: 4.6
Age rating: R
Genres: Horror
Runtime: 1h 20min
Director: Roxanne Benjamin

Title: Billionaire Boys Club
Released year: 2018
IMDb rating: 5.6
Age rating: R
Genres: Drama, Mystery & Thriller
Runtime: 1h 48min
Director: James Cox

Title: Code Name Banshee
Released year: 2022
IMDb rating: 3.6
Age rating: N/A
Genres: Mystery & Thriller, Action & Adventure
Runtime: 1h 33min
Direct

...

Genres: Comedy, Romance
Runtime: 1h 34min
Director: Chris Riedell

Title: Buddy Games
Released year: 2020
IMDb rating: 4.8
Age rating: R
Genres: Comedy
Runtime: 1h 36min
Director: Josh Duhamel

Title: The Night Stalker
Released year: 2016
IMDb rating: 5.8
Age rating: N/A
Genres: Mystery & Thriller, Crime, Drama
Runtime: 1h 29min
Director: Megan Griffiths

Title: Bad Hair
Released year: 2020
IMDb rating: 5.6
Age rating: N/A
Genres: Comedy, Horror, Music & Musical
Runtime: 1h 42min
Director: Justin Simien

Title: The Keeping Room
Released year: 2014
IMDb rating: 6.0
Age rating: R
Genres: Western, Drama
Runtime: 1h 35min
Director: Daniel Barber

Title: Borgman
Released year: 2013
IMDb rating: 6.7
Age rating: N/A
Genres: Mystery & Thriller, Drama, Horror, Made in Europe
Runtime: 1h 53min
Director: Alex van Warmerdam

Title: Hurricane Bianca
Released year: 2016
IMDb rating: 5.8
Age rating: N/A
Genres: Comedy, Drama
Runtime: 1h 24min
Director: Matt Kugelman

Title: It's Christmas, Eve
Releas

IMDb rating: 2.5
Age rating: N/A
Genres: Mystery & Thriller, Horror
Runtime: 1h 30min
Director: Jacob Johnston

Title: Trust
Released year: 2021
IMDb rating: 5.3
Age rating: N/A
Genres: Romance, Mystery & Thriller, Drama
Runtime: 1h 34min
Director: Brian DeCubellis

Title: Dogtanian and the Three Muskehounds
Released year: 2021
IMDb rating: 5.3
Age rating: N/A
Genres: Comedy, Animation, Romance, Kids & Family
Runtime: 1h 29min
Director: Toni García

Title: Manou the Swift
Released year: 2019
IMDb rating: 5.4
Age rating: G
Genres: Animation, Comedy, Kids & Family, Made in Europe
Runtime: 1h 28min
Director: Andrea Block

Title: The Last Stand
Released year: 2013
IMDb rating: 6.3
Age rating: R
Genres: Action & Adventure, Crime, Mystery & Thriller, Western, Made in Europe
Runtime: 1h 47min
Director: Kim Jee-woon

Title: Disturbing the Peace
Released year: 2020
IMDb rating: 3.3
Age rating: R
Genres: Action & Adventure, Western, Mystery & Thriller
Runtime: 1h 30min
Director: York Alec Shackl

Genres: Drama, Action & Adventure, Comedy
Runtime: 1h 32min
Director: Danny Lerner

Title: Dancing Through the Snow
Released year: 2021
IMDb rating: 5.2
Age rating: N/A
Genres: Comedy, Romance, Kids & Family
Runtime: 1h 25min
Director: Paul Shapiro

Title: Good Boy
Released year: 2020
IMDb rating: N/A
Age rating: N/A
Genres: Mystery & Thriller, Comedy, Horror
Runtime: 1h 29min
Director: Tyler MacIntyre

Title: Rabbit Academy: Mission Eggpossible
Released year: 2022
IMDb rating: 5.4
Age rating: N/A
Genres: Comedy, Animation, Kids & Family
Runtime: 1h 15min
Director: Ute von Münchow-Pohl

Title: The First Wave
Released year: 2021
IMDb rating: 7.0
Age rating: R
Genres: History, Documentary, Drama
Runtime: 1h 33min
Director: Matthew Heineman

Title: Modern Persuasion
Released year: 2020
IMDb rating: 4.9
Age rating: N/A
Genres: Comedy, Drama, Romance
Runtime: 1h 27min
Director: Alex Appel

Title: The Enchanted Christmas Cake
Released year: 2021
IMDb rating: 5.4
Age rating: G
Genres: Romance

Title: Christmas at the Drive-In
Released year: 2022
IMDb rating: 6.2
Age rating: G
Genres: Comedy, Romance
Runtime: 1h 25min
Director: Nicholas Bruckman

Title: The Christmas Candle
Released year: 2013
IMDb rating: 6.2
Age rating: PG
Genres: Drama, Kids & Family
Runtime: 1h 40min
Director: John Stephenson

Title: Pooka Lives!
Released year: 2020
IMDb rating: N/A
Age rating: R
Genres: Comedy, Horror
Runtime: 1h 20min
Director: Alejandro Brugués

Title: Sanak
Released year: 2021
IMDb rating: 6.7
Age rating: N/A
Genres: Mystery & Thriller, Action & Adventure
Runtime: 1h 57min
Director: Kanishk Varma

Title: Menorah in the Middle
Released year: 2022
IMDb rating: 4.6
Age rating: N/A
Genres: Drama, Romance, Mystery & Thriller, Kids & Family, Fantasy, Horror
Runtime: 1h 30min
Director: Jordan Kessler

Title: Color of Love
Released year: 2021
IMDb rating: N/A
Age rating: PG
Genres: Drama
Runtime: 1h 25min
Director: Don McBrearty

Title: Christmas Movie Magic
Released year: 2021
IMDb rating: 6

IMDb rating: N/A
Age rating: N/A
Genres: Horror
Runtime: 7min
Director: Elaine Mongeon

Title: The Ape Star
Released year: 2021
IMDb rating: 5.7
Age rating: PG
Genres: Animation, Kids & Family
Runtime: 1h 15min
Director: Linda Hambäck

Title: The State of Texas vs. Melissa
Released year: 2021
IMDb rating: 5.9
Age rating: N/A
Genres: Documentary
Runtime: 1h 42min
Director: Sabrina Van Tassel

Title: Hidden Mother
Released year: 2019
IMDb rating: N/A
Age rating: N/A
Genres: Horror
Runtime: 5min
Director: Joshua Erkman

Title: A Christmas Love Story
Released year: 2019
IMDb rating: 6.9
Age rating: N/A
Genres: Music & Musical, Romance, Drama
Runtime: 1h 24min
Director: Eric Close

Title: Materna
Released year: 2020
IMDb rating: 4.9
Age rating: N/A
Genres: Drama
Runtime: 1h 45min
Director: Eric Close

Title: Kate Berlant: Cinnamon in the Wind
Released year: 2022
IMDb rating: 6.3
Age rating: N/A
Genres: Comedy
Runtime: 44min
Director: Bo Burnham

Title: Guess Who's Coming to Christmas
Releas

In [13]:
# Return a dictionary with all the movie data
movie_data = {
    'title': Movies,
    'year': Released_year,
    'genre': genres,
    'runtime': runtimes,
    'rating': ratings_list,
    'age_rating': Age_Rating,
    'director_name':directors
}

# Convert to a DataFrame        
Hulu_df = pd.DataFrame(movie_data)

In [14]:
Hulu_df

,title,year,genre,runtime,rating,age_rating,director_name
0,Ghosted,2023,"Action & Adventure, Romance, Comedy",2h 0min,N/A,PG-13,Dexter Fletcher
1,The Meg,2018,"Science-Fiction, Action & Adventure, Horror, M...",1h 53min,5.6,PG-13,Jon Turteltaub
2,Boss Level,2021,"Mystery & Thriller, Science-Fiction, Action & ...",1h 34min,6.8,R,Joe Carnahan
3,Escape Room,2019,"Horror, Mystery & Thriller, Drama, Action & Ad...",1h 40min,6.4,PG-13,Adam Robitel
4,The Program,2015,"Drama, Sport, Made in Europe",1h 44min,6.5,PG-13,Stephen Frears
...,...,...,...,...,...,...,...
923,First Day,2017,Drama,17min,7.6,N/A,Julie Kalceff
924,Stolen by My Mother: The Kamiyah Mobley Story,2020,"Crime, Drama",1h 27min,6.4,N/A,N/A
925,The Secret Life of a Celebrity Surrogate,2020,Mystery & Thriller,1h 26min,5.2,N/A,N/A
926,Blippi Summer Games,2021,Kids & Family,40min,N/A,N/A,N/A


In [15]:
Hulu_df.to_csv('Hulu.csv',index = False)